In [ ]:
import pandas as pd
import calendar
from datetime import datetime
from tkinter import Tk, filedialog, StringVar, OptionMenu, Button, Label, Toplevel
import os
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.formatting.rule import CellIsRule

# -------------------------
# Helpers: file picker + year/month dialog (dropdowns)
# -------------------------
def ask_file():
    root = Tk()
    root.withdraw()
    path = filedialog.askopenfilename(
        title="Select ADWP Data File",
        filetypes=[("Excel files", "*.xlsx")]
    )
    root.destroy()
    return path

def ask_year_month():
    root = Tk()
    root.withdraw()
    result = {"year": None, "month": None}

    def on_ok():
        result["year"] = int(year_var.get())
        result["month"] = month_var.get()
        win.destroy()

    win = Toplevel(root)
    win.title("Select Year and Month")

    Label(win, text="Select Year:").grid(row=0, column=0, padx=10, pady=5)
    Label(win, text="Select Month:").grid(row=1, column=0, padx=10, pady=5)

    current_year = datetime.today().year
    years = [str(current_year - 1), str(current_year), str(current_year + 1)]
    year_var = StringVar(win)
    year_var.set(str(current_year))
    year_menu = OptionMenu(win, year_var, *years)
    year_menu.grid(row=0, column=1, padx=10, pady=5)

    months = list(calendar.month_name)[1:]
    month_var = StringVar(win)
    month_var.set(calendar.month_name[datetime.today().month])
    month_menu = OptionMenu(win, month_var, *months)
    month_menu.grid(row=1, column=1, padx=10, pady=5)

    Button(win, text="OK", command=on_ok).grid(row=2, column=0, columnspan=2, pady=10)
    win.wait_window()
    root.destroy()
    return result["year"], result["month"]

# -------------------------
# 1) File and selection
# -------------------------
file_path = ask_file()
if not file_path:
    raise SystemExit("No file selected.")
print("Selected file:", file_path)

sel_year, sel_month_name = ask_year_month()
sel_month = list(calendar.month_name).index(sel_month_name)
print(f"Selected Year={sel_year}, Month={sel_month_name} ({sel_month})")

selected_period = pd.Period(year=sel_year, month=sel_month, freq="M")

# -------------------------
# 2) Load & normalize Data sheet
# -------------------------
print("Reading Data sheet...")
df = pd.read_excel(file_path, sheet_name="Data", engine="openpyxl")

# Standardize Period Month to Period[M] and FTE numeric
df["Period Month"] = pd.to_datetime(df["Period Month"], errors="coerce").dt.to_period("M")
df["FTE"] = pd.to_numeric(df["FTE"], errors="coerce")

# -------------------------
# 3) Build 24-month horizon starting from Jan of selected year
#     (Jan-selected_year .. Dec-(selected_year+1))
# -------------------------
horizon = pd.period_range(start=f"{sel_year}-01", periods=24, freq="M")
month_labels = [p.strftime("%b-%y") for p in horizon]

# -------------------------
# 4) Grouping key (PID if present; else MPP ID). BF included so changes split rows.
# -------------------------
def build_key(row):
    pid = row.get("PID")
    mpp = row.get("MPP ID")
    gcb = row.get("GCB")
    bf = row.get("Business Framework")
    if pd.notna(pid) and str(pid).strip() != "":
        return ("PID", str(pid).strip(), str(gcb).strip() if pd.notna(gcb) else "", str(bf).strip() if pd.notna(bf) else "")
    else:
        return ("MPP", str(mpp).strip() if pd.notna(mpp) else "", str(gcb).strip() if pd.notna(gcb) else "", str(bf).strip() if pd.notna(bf) else "")

def snapshot_fte(series, sel_period):
    """Return snapshot FTE for the selected period:
       - If value exists at sel_period => return it
       - Else if there is future value => return first future value
       - Else if there is past value => return last past value
       - Else None
    """
    if len(series) == 0:
        return None
    if sel_period in series.index:
        return series.loc[sel_period]
    future = [p for p in series.index if p > sel_period]
    if future:
        return series.loc[future[0]]
    past = [p for p in series.index if p < sel_period]
    if past:
        return series.loc[past[-1]]
    return None

# -------------------------
# 5) Process each group and apply carry-forward per group (BF-scoped)
# -------------------------
print("Processing groups and applying carry-forward logic...")
grouped = df.groupby(df.apply(build_key, axis=1))
output_records = []

for key, group in grouped:
    kind = key[0]
    # sort by period month
    group = group.sort_values("Period Month", ignore_index=True)
    # series of month -> last FTE in that month
    s = group.groupby("Period Month", sort=True)["FTE"].last()

    # dimension columns: use latest row's dims for non-time fields
    latest = group.iloc[-1]
    record = {
        "Business Service L2": latest.get("Business Service L2"),
        "Business Framework Group": latest.get("Business Framework Group"),
        "Business Framework": latest.get("Business Framework"),
        "Country R1": latest.get("Country R1"),
        "Country R2": latest.get("Country R2"),
        "Country R3": latest.get("Country R3"),
        "PID": latest.get("PID") if kind == "PID" else None,
        "GCB": latest.get("GCB"),
        "MPP ID": latest.get("MPP ID") if kind == "MPP" else None,
        "Stack": latest.get("Stack") if kind == "MPP" else None
    }

    # carry-forward: iterate horizon chronologically and carry last known FTE (including -1)
    last_fte = None
    for p in horizon:
        lbl = p.strftime("%b-%y")
        if p in s.index:
            val = s.loc[p]
            # treat numeric values (including -1) as valid; NaN means no change
            if pd.notna(val):
                last_fte = val
        record[lbl] = last_fte if last_fte is not None else None

    # snapshot FTE: use the series s to derive selected snapshot using rules above
    record["FTE"] = snapshot_fte(s, selected_period)

    output_records.append(record)

# -------------------------
# 6) Build DataFrame, order columns and sort
# -------------------------
out_df = pd.DataFrame(output_records)

# Ensure dims + FTE + MPP/Stack then months (keep only existing columns)
dims = [
    "Business Service L2", "Business Framework Group", "Business Framework",
    "Country R1", "Country R2", "Country R3", "PID", "GCB",
    "FTE", "MPP ID", "Stack"
]
dims = [c for c in dims if c in out_df.columns]
ordered_cols = dims + [c for c in month_labels if c in out_df.columns]
out_df = out_df[ordered_cols]

# Sorting
sort_cols = [
    "Business Service L2", "Business Framework Group", "Business Framework",
    "Country R1", "Country R2", "Country R3", "PID", "MPP ID", "GCB"
]
existing_sort_cols = [c for c in sort_cols if c in out_df.columns]
if existing_sort_cols:
    out_df = out_df.sort_values(existing_sort_cols, na_position="last").reset_index(drop=True)

# -------------------------
# 7) Save Output to Excel
# -------------------------
out_file = os.path.join(os.path.dirname(file_path), f"ADWP_Output_{sel_year}_{sel_month:02}.xlsx")
with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
    out_df.to_excel(writer, sheet_name="Output", index=False)

# -------------------------
# 8) Apply conditional formatting: negative numbers -> red font
# -------------------------
wb = load_workbook(out_file)
ws = wb["Output"]

max_row = ws.max_row
max_col = ws.max_column
col_letters = [ws.cell(row=1, column=i).column_letter for i in range(1, max_col + 1)]

red_font = Font(color="FF0000")
for col_letter in col_letters:
    ws.conditional_formatting.add(
        f"{col_letter}2:{col_letter}{max_row}",
        CellIsRule(operator='lessThan', formula=['0'], font=red_font)
    )

wb.save(out_file)

print(f"✅ ADWP Output saved: {out_file}")
print(f"Rows: {len(out_df):,}, Columns: {len(out_df.columns):,}")


In [ ]:
import pandas as pd
import calendar
from datetime import datetime
from tkinter import Tk, filedialog, StringVar, OptionMenu, Button, Label, Toplevel
import os
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.formatting.rule import CellIsRule


# -------------------------
# Step 0. Helpers
# -------------------------
def ask_file():
    root = Tk()
    root.withdraw()
    path = filedialog.askopenfilename(
        title="Select ADWP Data File",
        filetypes=[("Excel files", "*.xlsx")]
    )
    root.destroy()
    return path

def ask_year_month():
    root = Tk()
    root.withdraw()
    result = {"year": None, "month": None}

    def on_ok():
        result["year"] = int(year_var.get())
        result["month"] = month_var.get()
        win.destroy()

    win = Toplevel(root)
    win.title("Select Year and Month")

    Label(win, text="Select Year:").grid(row=0, column=0, padx=10, pady=5)
    Label(win, text="Select Month:").grid(row=1, column=0, padx=10, pady=5)

    current_year = datetime.today().year
    years = [str(current_year - 1), str(current_year), str(current_year + 1)]
    year_var = StringVar(win)
    year_var.set(str(current_year))
    year_menu = OptionMenu(win, year_var, *years)
    year_menu.grid(row=0, column=1, padx=10, pady=5)

    months = list(calendar.month_name)[1:]
    month_var = StringVar(win)
    month_var.set(calendar.month_name[datetime.today().month])
    month_menu = OptionMenu(win, month_var, *months)
    month_menu.grid(row=1, column=1, padx=10, pady=5)

    Button(win, text="OK", command=on_ok).grid(row=2, column=0, columnspan=2, pady=10)
    win.wait_window()
    root.destroy()
    return result["year"], result["month"]


# -------------------------
# Step 1. Pick file + year/month
# -------------------------
file_path = ask_file()
if not file_path:
    raise SystemExit("No file selected.")
print("Selected file:", file_path)

sel_year, sel_month_name = ask_year_month()
sel_month = list(calendar.month_name).index(sel_month_name)
print(f"Selected Year={sel_year}, Month={sel_month_name} ({sel_month})")

selected_period = pd.Period(year=sel_year, month=sel_month, freq="M")

# -------------------------
# Step 2. Read input & normalize months
# -------------------------
print("Reading Data sheet...")
df = pd.read_excel(file_path, sheet_name="Data", engine="openpyxl")

df["Period Month"] = pd.to_datetime(df["Period Month"], errors="coerce").dt.to_period("M")
df["FTE"] = pd.to_numeric(df["FTE"], errors="coerce")

# -------------------------
# Step 3. Build 24-month horizon
# -------------------------
horizon = pd.period_range(start=selected_period, periods=24, freq="M")
month_labels = [p.strftime("%b-%y") for p in horizon]


# -------------------------
# Step 4. Grouping
# -------------------------
def build_key(row):
    pid = row.get("PID")
    mpp = row.get("MPP ID")
    gcb = row.get("GCB")
    bf = row.get("Business Framework")
    if pd.notna(pid) and str(pid).strip() != "":
        return ("PID", str(pid).strip(), str(gcb).strip() if pd.notna(gcb) else "", str(bf).strip() if pd.notna(bf) else "")
    else:
        return ("MPP", str(mpp).strip() if pd.notna(mpp) else "", str(gcb).strip() if pd.notna(gcb) else "", str(bf).strip() if pd.notna(bf) else "")


def snapshot_fte(series, sel_period):
    """Get FTE snapshot value for the selected period."""
    if sel_period in series.index:
        return series.loc[sel_period]
    future_periods = [p for p in series.index if p > sel_period]
    if future_periods:
        return series.loc[future_periods[0]]
    past_periods = [p for p in series.index if p < sel_period]
    if past_periods:
        return series.loc[past_periods[-1]]
    return None


print("Processing groups...")
grouped = df.groupby(df.apply(build_key, axis=1))
output_records = []

for key, group in grouped:
    kind = key[0]
    group = group.sort_values("Period Month", ignore_index=True)
    s = group.groupby("Period Month", sort=True)["FTE"].last()

    latest = group.iloc[-1]
    record = {
        "Business Service L2": latest.get("Business Service L2"),
        "Business Framework Group": latest.get("Business Framework Group"),
        "Business Framework": latest.get("Business Framework"),
        "Country R1": latest.get("Country R1"),
        "Country R2": latest.get("Country R2"),
        "Country R3": latest.get("Country R3"),
        "PID": latest.get("PID") if kind == "PID" else None,
        "GCB": latest.get("GCB"),
        "MPP ID": latest.get("MPP ID") if kind == "MPP" else None,
        "Stack": latest.get("Stack") if kind == "MPP" else None
    }

    # Initialize horizon
    last_fte = None
    for p in horizon:
        lbl = p.strftime("%b-%y")
        if p in s.index:
            val = s.loc[p]
            if pd.notna(val):
                last_fte = val
        record[lbl] = last_fte if last_fte is not None else None

    # Snapshot FTE for selected period
    record["FTE"] = snapshot_fte(s, selected_period)

    output_records.append(record)


# -------------------------
# Step 5. Build output dataframe, sort
# -------------------------
out_df = pd.DataFrame(output_records)

dims = [
    "Business Service L2", "Business Framework Group", "Business Framework",
    "Country R1", "Country R2", "Country R3", "PID", "GCB",
    "FTE", "MPP ID", "Stack"
]
dims = [c for c in dims if c in out_df.columns]
ordered_cols = dims + [c for c in month_labels if c in out_df.columns]
out_df = out_df[ordered_cols]

sort_cols = [
    "Business Service L2", "Business Framework Group", "Business Framework",
    "Country R1", "Country R2", "Country R3", "PID", "MPP ID", "GCB"
]
existing_sort_cols = [c for c in sort_cols if c in out_df.columns]
if existing_sort_cols:
    out_df = out_df.sort_values(existing_sort_cols, na_position="last").reset_index(drop=True)

# -------------------------
# Step 6. Save output to Excel
# -------------------------
out_file = os.path.join(os.path.dirname(file_path), f"ADWP_Output_{sel_year}_{sel_month:02}.xlsx")
with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
    out_df.to_excel(writer, sheet_name="Output", index=False)

# -------------------------
# Step 7. Apply conditional formatting (negative → red text)
# -------------------------
wb = load_workbook(out_file)
ws = wb["Output"]

max_row = ws.max_row
max_col = ws.max_column
col_letters = [ws.cell(row=1, column=i).column_letter for i in range(1, max_col + 1)]

red_font = Font(color="FF0000")

for col_letter in col_letters:
    ws.conditional_formatting.add(
        f"{col_letter}2:{col_letter}{max_row}",
        CellIsRule(operator="lessThan", formula=["0"], font=red_font)
    )

wb.save(out_file)
print(f"✅ ADWP Output saved: {out_file}")
print(f"Rows: {len(out_df):,}, Columns: {len(out_df.columns):,}")


In [ ]:
import pandas as pd
import calendar
from datetime import datetime
from tkinter import Tk, filedialog, StringVar, OptionMenu, Button, Label, Toplevel
import os

# -------------------------
# Step 1. File selection
# -------------------------
root = Tk()
root.withdraw()  # hide root window

file_path = filedialog.askopenfilename(
    title="Select ADWP Data File",
    filetypes=[("Excel files", "*.xlsx")]
)
if not file_path:
    raise SystemExit("❌ No file selected.")
print(f"Selected file: {file_path}")


# -------------------------
# Step 2. Custom dialog for Year & Month
# -------------------------
def ask_year_month():
    result = {"year": None, "month": None}

    def on_ok():
        result["year"] = year_var.get()
        result["month"] = month_var.get()
        win.destroy()

    win = Toplevel(root)
    win.title("Select Year and Month")

    Label(win, text="Select Year:").grid(row=0, column=0, padx=10, pady=5)
    Label(win, text="Select Month:").grid(row=1, column=0, padx=10, pady=5)

    # Year dropdown
    current_year = datetime.today().year
    years = [str(current_year - 1), str(current_year), str(current_year + 1)]
    year_var = StringVar(win)
    year_var.set(str(current_year))  # default
    year_menu = OptionMenu(win, year_var, *years)
    year_menu.grid(row=0, column=1, padx=10, pady=5)

    # Month dropdown
    months = list(calendar.month_name)[1:]
    month_var = StringVar(win)
    month_var.set(calendar.month_name[datetime.today().month])  # default
    month_menu = OptionMenu(win, month_var, *months)
    month_menu.grid(row=1, column=1, padx=10, pady=5)

    Button(win, text="OK", command=on_ok).grid(row=2, column=0, columnspan=2, pady=10)

    win.wait_window()  # wait for user to close

    return int(result["year"]), result["month"]


selected_year, selected_month = ask_year_month()
print(f"Selected: {selected_month} {selected_year}")


# -------------------------
# Step 3. Processing
# -------------------------
print("⏳ Reading input file...")
df = pd.read_excel(file_path, sheet_name="Data", engine="openpyxl")
df["Period Month"] = pd.to_datetime(df["Period Month"]).dt.to_period("M").dt.to_timestamp()

# Current selection
sel_year = selected_year
sel_month = list(calendar.month_name).index(selected_month)
curr_date = datetime(sel_year, sel_month, 1)

# Horizon (24 months from Jan of selected year)
start_date = datetime(sel_year, 1, 1)
horizon = pd.date_range(start=start_date, periods=24, freq="MS")
month_labels = [d.strftime("%b-%y") for d in horizon]

output_records = []

def get_key(row):
    if pd.notna(row["PID"]):
        return ("PID", row["PID"], row["GCB"], row["Business Framework"])
    else:
        return ("MPP", row["MPP ID"], row["GCB"], row["Business Framework"])

print("⏳ Processing records...")
grouped = df.groupby(df.apply(get_key, axis=1))

for key, group in grouped:
    kind = key[0]
    group = group.sort_values("Period Month")

    latest = group.iloc[-1]
    record = {
        "Business Service L2": latest.get("Business Service L2"),
        "Business Framework Group": latest.get("Business Framework Group"),
        "Business Framework": latest.get("Business Framework"),
        "Country R1": latest.get("Country R1"),
        "Country R2": latest.get("Country R2"),
        "Country R3": latest.get("Country R3"),
        "PID": latest.get("PID") if kind == "PID" else None,
        "GCB": latest.get("GCB"),
        "MPP ID": latest.get("MPP ID") if kind == "MPP" else None,
        "Stack": latest.get("Stack") if kind == "MPP" else None
    }

    # initialize horizon columns
    for label in month_labels:
        record[label] = None

    last_fte = None
    for month in month_labels:
        month_dt = datetime.strptime(month, "%b-%y")
        row = group[group["Period Month"] == month_dt]
        if not row.empty:
            last_fte = row["FTE"].iloc[0]
        if last_fte is not None:
            record[month] = last_fte
        if month_dt <= curr_date and record[month] is not None:
            record["FTE"] = record[month]

    output_records.append(record)

# Build DataFrame
out_df = pd.DataFrame(output_records)

# Sorting
sort_cols = [
    "Business Service L2", "Business Framework Group", "Business Framework",
    "Country R1", "Country R2", "Country R3", "PID", "MPP ID", "GCB"
]
out_df = out_df.sort_values(sort_cols, na_position="last").reset_index(drop=True)

# Save output
out_file = os.path.join(
    os.path.dirname(file_path),
    f"ADWP_Output_{sel_year}_{sel_month:02}.xlsx"
)
with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
    out_df.to_excel(writer, sheet_name="Output", index=False)

print(f"✅ ADWP Output written to: {out_file}")
print(f"Rows: {len(out_df):,}, Columns: {len(out_df.columns)}")
